# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
my_cities_to_load = 'C:\\Users\\me\\Desktop\\BBC\\COPY_but-ind-data-pt-06-2020-u-c\\Unit_06_Python_APIs\\Homework\\WeatherPy\\output_data\\my_cities\\my_cities.csv'
my_cities = pd.read_csv(my_cities_to_load)
my_cities.dropna(inplace = True) 
my_cities.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,kruisfontein,-34.00,24.73,77.74,18.0,0.0,14.70,ZA,1.595331e+09
1,tshikapa,-6.42,20.80,97.52,13.0,0.0,3.69,CD,1.595331e+09
2,airai,-8.93,125.41,56.77,100.0,27.0,3.42,TL,1.595331e+09
3,souillac,-20.52,57.52,73.40,49.0,75.0,13.87,MU,1.595331e+09
4,bengkulu,-3.80,102.27,81.93,80.0,100.0,6.38,ID,1.595330e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = my_cities[["Latitude", "Longitude"]]
weight = my_cities["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                  dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_one = my_cities.loc[(my_cities["Max Temperature"] < 80)]
ideal_temp = ideal_one.loc[(ideal_one["Max Temperature"] > 70)]
ideal_wind = my_cities.loc[(my_cities["Wind Speed"] < 10)]
ideal_clouds =  my_cities.loc[(my_cities["Cloudiness"] < 1)]

In [6]:
ideal_wind_clouds = pd.merge(ideal_wind, ideal_clouds, on=["City", "City"])

In [7]:
ideal_conditions = pd.merge(ideal_temp, ideal_wind_clouds, on=["City", "City"])
ideal_conditions.drop(ideal_conditions.iloc[:, 8:25], inplace = True, axis = 1) 
ideal_conditions

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country
0,manyana,-23.40,21.72,70.29,22.0,0.0,8.88,BW
1,camacupa,-12.02,17.48,79.43,22.0,0.0,4.21,AO
2,hermanus,-34.42,19.23,75.00,34.0,0.0,3.00,ZA
3,plettenberg bay,-34.05,23.37,78.26,22.0,0.0,4.94,ZA
4,xai-xai,-25.05,33.64,75.24,47.0,0.0,5.01,MZ
5,koslan,63.46,48.90,75.04,40.0,0.0,9.78,RU
6,ekuvukeni,-28.47,30.16,74.14,16.0,0.0,2.17,ZA
7,bredasdorp,-34.53,20.04,73.40,38.0,0.0,5.82,ZA
8,inta,66.03,60.17,75.07,43.0,0.0,4.21,RU
9,arraial do cabo,-22.97,-42.03,71.60,83.0,0.0,8.05,BR


In [8]:
ideal_conditions.drop(ideal_conditions.iloc[:, 3:7], inplace = True, axis = 1) 
ideal_conditions = ideal_conditions.drop([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
ideal_conditions['Hotel'] = ""
ideal_conditions = ideal_conditions[["City","Country","Latitude","Longitude", "Hotel"]]
hotel_df = ideal_conditions
hotel_df

,City,Country,Latitude,Longitude,Hotel
0,manyana,BW,-23.40,21.72,
1,camacupa,AO,-12.02,17.48,
2,hermanus,ZA,-34.42,19.23,
3,plettenberg bay,ZA,-34.05,23.37,
4,xai-xai,MZ,-25.05,33.64,
5,koslan,RU,63.46,48.90,
6,ekuvukeni,ZA,-28.47,30.16,
7,bredasdorp,ZA,-34.53,20.04,
8,inta,RU,66.03,60.17,
9,arraial do cabo,BR,-22.97,-42.03,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
